In [1]:
# traing model
# import libary

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, LeakyReLU, Conv2DTranspose, Activation, Reshape
from keras.layers import Conv2D,Input
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.backend.tensorflow_backend import set_session

parentPath = os.path.abspath("../GanLossLib")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

from ganlosslib import *

K.set_learning_phase(False) 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
# 設定 Keras 使用的 Session

Using TensorFlow backend.


In [ ]:
class Dcgan():
    
    def __init__(self):
        
        self.d_depth = 64
        self.g_depth = 512
        self.g_dim = 4
        self.alpha = 0.2
        self.lr = 1e-4
        self.batch = 64
        self.trainsteps = 1000
        self.imageshape = (64, 64, 3)
        self.noiseshape = (100,)
        self.truncateNormal = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.02, seed=None)
        self.randomNormal = keras.initializers.RandomNormal(mean=0.0, stddev=0.02, seed=None)
        
        self.Discriminator()
        self.Generator()
        self.get_function()
        
    
    def Discriminator(self):
        
        self.discriminator = Sequential()
        input_shape = (64, 64, 3)
        self.discriminator.add(Conv2D(self.d_depth*1, 5, strides=2, kernel_initializer=self.truncateNormal, input_shape=input_shape, padding='same'))
    #    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
        self.discriminator.add(LeakyReLU(self.alpha))

        self.discriminator.add(Conv2D(self.d_depth*2, 5, strides=2, kernel_initializer=self.truncateNormal, padding='same'))
    #    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
        self.discriminator.add(LeakyReLU(self.alpha))

        self.discriminator.add(Conv2D(self.d_depth*4, 5, strides=2, kernel_initializer=self.truncateNormal, padding='same'))
    #    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
        self.discriminator.add(LeakyReLU(self.alpha))

        self.discriminator.add(Conv2D(self.d_depth*8, 5, strides=2, kernel_initializer=self.truncateNormal, padding='same'))
    #    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
        self.discriminator.add(LeakyReLU(self.alpha))

        # Out: 1-dim probability
        self.discriminator.add(Flatten())
        self.discriminator.add(Dense(1))
        
    def Generator(self):
        
        self.generator = Sequential()
        self.generator.add(Dense(self.g_dim*self.g_dim*self.g_depth, kernel_initializer=self.truncateNormal, input_dim=100))
        self.generator.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
        self.generator.add(Reshape((self.g_dim, self.g_dim, self.g_depth)))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        self.generator.add(Conv2DTranspose(int(self.g_depth/2), 5, kernel_initializer=self.truncateNormal, strides=2, padding='same'))
        self.generator.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
        self.generator.add(Activation('relu'))

        self.generator.add(Conv2DTranspose(int(self.g_depth/4), 5, kernel_initializer=self.truncateNormal, strides=2, padding='same'))
        self.generator.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
        self.generator.add(Activation('relu'))

        self.generator.add(Conv2DTranspose(int(self.g_depth/8), 5, kernel_initializer=self.truncateNormal, strides=2, padding='same'))
        self.generator.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
        self.generator.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.generator.add(Conv2DTranspose(3, 5, strides=2, kernel_initializer=self.truncateNormal, padding='same'))
        self.generator.add(Activation('tanh'))
        
    def get_function(self):
        
        opt = Adam(lr=self.lr, beta_1=0.5, beta_2=0.9)
        self.d_train, self.g_train = gan_loss(self.imageshape, self.noiseshape, self.generator, self.discriminator, K, opt)
        
    def train(self, datasets):
        print("training...")
        for k in range(self.trainsteps):
            for kk in range(len(datasets)):
                dataset = np.load(datasets[kk])
                ite = 0
                while ite<len(dataset):
                    data = dataset[ite:(ite+self.batch)]
                    noise = np.random.normal(0, 1.0, size=[len(data), 100])
                    ϵ = np.random.uniform(size=(self.batch, 1, 1 ,1))
                    for l in range(1):
                        errD,  = self.d_train([data, noise, ϵ])
                        errD = np.mean(errD)

                    for l in range(2):
                        errG, = self.g_train([noise])
                        errG = np.mean(errG)

                    print(errD, errG)
                    ite+=self.batch
                
                print("save")
                noise = np.random.normal(0, 1.0, size=[36, 100])
                images_fake = self.generator.predict(noise)
                width = 6
                new_im = Image.new('RGB', (64*width,64*width))
                for ii in range(width):
                    for jj in range(width):
                        index=ii*width+jj
                        image = (images_fake[index]/2+0.5)*255
                        image = image.astype(np.uint8)
                        new_im.paste(Image.fromarray(image,"RGB"), (64*ii,64*jj))
                filename = "images/fakeFace%d.png"%k
                new_im.save(filename)
                self.generator.save("models/generator%d.h5"%k)
                self.discriminator.save("models/discriminator%d.h5"%k)

In [ ]:
dcgan = Dcgan()

celebas = [None]*5
for i in range(5):
    celebas[i] = '../../celeba%d.npy'%i

dcgan.train(celebas)

training...
1.3717 0.692056
1.30449 0.689118
1.22283 0.677852
1.10563 0.642612
0.993664 0.577224
0.982533 0.548656
0.985347 0.606412
0.879037 0.698536
0.758788 0.767074
0.692818 0.807395
0.642992 0.828087
0.611812 0.836736
0.609935 0.827724
0.609965 0.813047
0.625828 0.792995
0.636403 0.774865
0.66237 0.764657
0.698517 0.756808
